In [1]:
from aip import AipNlp
from tqdm import tqdm
import json
import time

In [2]:
APP_ID = '11643098'
API_KEY = '7eKRtXao1UE5pitgUDmBS0Bb'
SECRET_KEY = 'uRObC4UnpqYUaQGExsGnX1DM8PRZNQbL'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [3]:
t = '2014年09月25日01:30接110指令，禹州市法院小区1号楼1单元14层1401号有人包被抢了，杨文琼称今天晚上和张修兵一起在市中区凤凰路中段丽景豆花饭店对面的小巷被两人持刀抢劫，被抢财物有两个包，一共有2300元现金，一部蓝色酷派手机（购于2014年1月11日，购价900元）以及信用卡，身份证等若干证件。'

In [4]:
json_str = json.dumps(client.lexer(t))
json.loads(json_str)

{'items': [{'basic_words': ['2014', '年', '0', '9', '月', '25', '日'],
   'byte_length': 14,
   'byte_offset': 0,
   'formal': '',
   'item': '2014年09月25日',
   'loc_details': [],
   'ne': 'TIME',
   'pos': '',
   'uri': ''},
  {'basic_words': ['01', ':', '30'],
   'byte_length': 5,
   'byte_offset': 14,
   'formal': '',
   'item': '01:30',
   'loc_details': [],
   'ne': 'TIME',
   'pos': '',
   'uri': ''},
  {'basic_words': ['接'],
   'byte_length': 2,
   'byte_offset': 19,
   'formal': '',
   'item': '接',
   'loc_details': [],
   'ne': '',
   'pos': 'v',
   'uri': ''},
  {'basic_words': ['110'],
   'byte_length': 3,
   'byte_offset': 21,
   'formal': '',
   'item': '110',
   'loc_details': [],
   'ne': '',
   'pos': 'm',
   'uri': ''},
  {'basic_words': ['指令'],
   'byte_length': 4,
   'byte_offset': 24,
   'formal': '',
   'item': '指令',
   'loc_details': [],
   'ne': '',
   'pos': 'n',
   'uri': ''},
  {'basic_words': ['，'],
   'byte_length': 2,
   'byte_offset': 28,
   'formal': '',
   '

#### RMRB解析

In [3]:
import pickle
with open('./RMRB_text', 'rb') as inp:
    text = pickle.load(inp)

In [5]:
len(text)

435393

In [6]:
loc = []
for i in text:
    for j in ['单元','小区','路','街','巷','区','市','省','城','楼','栋','县','村','乡','镇','组']:
        if j in i:
            loc.append(i)
            break

In [7]:
len(loc)

159181

In [8]:
xx = []
yy = []

for i in tqdm(range(len(loc))):
    time.sleep(0.2)
    x = []
    y = []
    
    try:         
        json_str = json.dumps(client.lexer(loc[i]))
        dt = json.loads(json_str)
    except:
        time.sleep(300)
        continue
              
    try: 
        for i in dt['items']:
            js = json.dumps(i)
            cont = json.loads(js)
            x.append(cont['item'])
            y.append(cont['ne'])
    except:
        time.sleep(1)
        continue
        
    xx.append(x)
    yy.append(y)

100%|███████████████████████████████████████████████████████████████████████| 159181/159181 [41:02:45<00:00,  1.08it/s]


In [10]:
print(xx[0])

['人民网', '北京', '1月2日', '电', '据', '中央纪委监察部', '网站', '消息', '，', '日前', '，', '经', '中共中央', '批准', '，', '中共中央纪委', '对', '湖南省政协', '原', '副主席', '童名谦', '严重', '违纪', '违法', '问题', '进行', '了', '立案', '检查', '。']


In [11]:
print(yy[0])

['', 'LOC', 'TIME', '', '', '', '', '', '', '', '', '', 'ORG', '', '', 'ORG', '', 'ORG', '', '', 'PER', '', '', '', '', '', '', '', '', '']


#### JYAQ解析

In [3]:
path = 'D:/workspace/nertools/test/jyaq.csv'
with open(path, 'r' ,encoding='utf-8') as fp:
    sents = []
    labels = []
    
    for line in tqdm(fp.readlines()):
        time.sleep(0.2)
        if len(line) > 3:
            line = line.strip().strip('"').replace(' ','')

            json_str = json.dumps(client.lexer(line))
            dt = json.loads(json_str)

            sent = []
            label = []
            for i in dt['items']:
                js = json.dumps(i)
                cont = json.loads(js)
                sent.append(cont['item'])
                label.append(cont['ne'])

            sents.append(sent)
            labels.append(label)

  1%|▋                                                                                | 9/1021 [00:05<10:48,  1.56it/s]


KeyboardInterrupt: 

In [4]:
print(sents[56])

['2017年', '5月16日23时30分', '许', '，', '阿牛', '么子呷', '报警', '称', '自己', '在', '位于', '川主寺巷20号', '门市', '内', '休息', '是', '被', '5名', '男子', '进门', '市', '来', '持', '刀', '威胁', '抢劫', '走', '了', '500元', '现金', '。']


In [5]:
print(labels[56])

['TIME', 'TIME', '', '', 'PER', '', '', '', '', '', '', 'LOC', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [13]:
def f(x):
    if x=='':
        return 'O'
    if x=='TIME':
        return 'TIM'
    else:
        return x

In [14]:
ll=[]
for i in yy:
    ll.append(list(map(f,i)))

In [15]:
yy = ll 

In [25]:
from collections import Counter
lb_all = []
for l in ll:
    lb_all.extend(l)
counts = Counter(lb_all)
counts

Counter({'B-LOC': 235524,
         'B-ORG': 125394,
         'B-PER': 161718,
         'B-TIM': 185785,
         'I-LOC': 443006,
         'I-ORG': 581948,
         'I-PER': 283164,
         'I-TIM': 541976,
         'O': 15689756})

In [24]:
ss = []
ll = []
for i in range(len(xx)):
    s = []
    l = []
    for j in range(len(xx[i])):
        s.extend(list(xx[i][j]))
        if yy[i][j] == 'O':
            l.extend([yy[i][j]]*(len(xx[i][j])))
        else:
            l.append('B-'+yy[i][j])
            l.extend(['I-'+yy[i][j]]*(len(xx[i][j])-1))
    ss.append(s)
    ll.append(l)

In [26]:
import pickle
with open('./RMRB_from_baidu_label', 'wb') as outp:
            pickle.dump(ss, outp)
            pickle.dump(ll, outp)